# Projet maison n° 3

In [2]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [601]:
# names us
def df_names_us():
    
    df = pd.DataFrame(columns=['year','name', 'gender', 'births'])

    for i in range(1880,2021):
        df1 = pd.read_csv(('names/yob{}.txt').format(i), names=['name', 'gender', 'births'])
        df1['year'] = i 
        df = df.append(df1)
        
    df.reset_index(inplace=True)
    df.drop(columns=['index'], inplace=True)
 
    return df

In [596]:
%%time
df = df_names_us()

CPU times: user 6.02 s, sys: 2.45 s, total: 8.47 s
Wall time: 8.75 s


In [597]:
df

,index,year,name,gender,births
0,0,1880,Mary,F,7065
1,1,1880,Anna,F,2604
2,2,1880,Emma,F,2003
3,3,1880,Elizabeth,F,1939
4,4,1880,Minnie,F,1746
...,...,...,...,...,...
2020858,31266,2020,Zykell,M,5
2020859,31267,2020,Zylus,M,5
2020860,31268,2020,Zymari,M,5
2020861,31269,2020,Zyn,M,5


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [586]:
# names fr
def df_names_fr():
    
    df = pd.read_csv('nat2020.csv', sep=';')
    df = df.rename(columns={'sexe': 'gender','annais': 'year','preusuel': 'name','nombre': 'births'})
    columns=['year','name', 'gender', 'births']
    df = df[columns]
    
    df = df.dropna()
    df = df.drop(df.loc[df['name']=='_PRENOMS_RARES'].index)
    df = df.drop(df.loc[df['year']=='XXXX'].index)
    df = df.drop(df.loc[df['name'].apply(len) < 2].index)
    df['gender'] = df['gender'].map({1:'M',2:'F'})
    df['name'] = df['name'].apply(str.capitalize)
    
    df.sort_values(['year','gender','births','name'], ascending=[True, True, False, True], inplace=True)
    df.reset_index(inplace=True)
    df.drop(columns=['index'], inplace=True)
    
    return df

In [587]:
df2 = df_names_fr()
df2

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696
...,...,...,...,...
630401,2020,Élouen,M,3
630402,2020,Énes,M,3
630403,2020,Éphraïm,M,3
630404,2020,Éros,M,3


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [584]:
# taux de change
def df_taux_change(devises):
    
    df = pd.read_csv('Webstat_Export_20210929.csv', sep=';', na_values=['-'])
    df = df.dropna()
    df = df.drop(df.index[[0,1,2,3,4]])
    df = df.rename(columns = lambda x: x[-4:-1])
    df = df.rename(columns= {'re ':'Date'})
    df = df.set_index('Date')
    df.index = pd.to_datetime(df.index)
    df = df.replace(',','.',regex=True).astype(float)
    
    df = df[devises]
    return df

In [585]:
df3 = df_taux_change(['CHF', 'GBP', 'USD'])
df3

,CHF,GBP,USD
Date,,,
2006-12-29,1.6069,0.67150,1.3170
2006-12-28,1.6058,0.67115,1.3173
2006-12-27,1.6058,0.67085,1.3159
2006-12-22,1.6029,0.67150,1.3192
2006-12-21,1.6032,0.67160,1.3178
...,...,...,...
2000-07-25,1.5538,0.62010,0.9410
2000-07-24,1.5555,0.61510,0.9332
2000-07-21,1.5511,0.61850,0.9363


### Tests

In [602]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630406)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [603]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [604]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 10.313s

OK
